In [ ]:
import importlib
import supporting_functions
importlib.reload(supporting_functions)
from supporting_functions import *
from cryptanalysis import *

In [139]:
possible = {}
for i in range(8):
    possible[i] = {'e':[], 'a':[]}

for b in range(8):
    
    plain_texts = []
    for i in range(128):
        p = "ff"*(7-b)
        p += int_to_str(i)
        p += "ff"*(b)
        plain_texts.append(p)
    cipher_texts = encode_multiple(plain_texts)
    
    for i in range(128):
        a = int_to_poly(i)
        for j in range(1,127):
            flag = True
            e = j
            for ix in range(len(cipher_texts)):
                if b!=0:
                    p = plain_texts[ix][-2*(b+1):-2*(b)]
                    c = cipher_texts[ix][-2*(b+1):-2*(b)]
                else:
                    p = plain_texts[ix][-2*(b+1):]
                    c = cipher_texts[ix][-2*(b+1):]
                
                ans = pair_to_poly(p)
                # E
                ans = power_poly(ans,e)
                # A
                ans = mult_poly(ans,a)
                # E
                ans = power_poly(ans,e)
                # A
                ans = mult_poly(ans,a)
                # E
                ans = power_poly(ans,e)
                ans = poly_to_pair(ans)
                
                if ans!=c:
                    flag = False
                    break
                
            if flag:
                possible[b]['a'].append(i)
                possible[b]['e'].append(j)
    
    print(b)
    print(possible)

0
{0: {'e': [25, 94, 8], 'a': [38, 71, 126]}, 1: {'e': [], 'a': []}, 2: {'e': [], 'a': []}, 3: {'e': [], 'a': []}, 4: {'e': [], 'a': []}, 5: {'e': [], 'a': []}, 6: {'e': [], 'a': []}, 7: {'e': [], 'a': []}}
1
{0: {'e': [25, 94, 8], 'a': [38, 71, 126]}, 1: {'e': [19, 107, 1], 'a': [27, 27, 52]}, 2: {'e': [], 'a': []}, 3: {'e': [], 'a': []}, 4: {'e': [], 'a': []}, 5: {'e': [], 'a': []}, 6: {'e': [], 'a': []}, 7: {'e': [], 'a': []}}
2
{0: {'e': [25, 94, 8], 'a': [38, 71, 126]}, 1: {'e': [19, 107, 1], 'a': [27, 27, 52]}, 2: {'e': [48, 23, 56], 'a': [11, 100, 122]}, 3: {'e': [], 'a': []}, 4: {'e': [], 'a': []}, 5: {'e': [], 'a': []}, 6: {'e': [], 'a': []}, 7: {'e': [], 'a': []}}
3
{0: {'e': [25, 94, 8], 'a': [38, 71, 126]}, 1: {'e': [19, 107, 1], 'a': [27, 27, 52]}, 2: {'e': [48, 23, 56], 'a': [11, 100, 122]}, 3: {'e': [116, 45, 93], 'a': [57, 73, 112]}, 4: {'e': [], 'a': []}, 5: {'e': [], 'a': []}, 6: {'e': [], 'a': []}, 7: {'e': [], 'a': []}}
4
{0: {'e': [25, 94, 8], 'a': [38, 71, 126]}, 

In [148]:
soln = {}
for i in range(8):
    soln[i] = {'a' : set(), 'e' : set()}
    
for b in range(1,8):
    
    plain_texts = []
    for i in range(128):
        p = "ff"*(7-b)
        p += int_to_str(i)
        p += "ff"*(b)
        plain_texts.append(p)
    cipher_texts = encode_multiple(plain_texts)
    
    for i in range(128):
        a21 = int_to_poly(i)
        for j in range(len(possible[b-1]['a'])):
            a11 = int_to_poly(possible[b-1]['a'][j])
            e1 = possible[b-1]['e'][j]
            for k in range(len(possible[b]['a'])):
                a22 = int_to_poly(possible[b]['a'][k])
                e2 = possible[b]['e'][k]
                flag = True
                for ix in range(len(cipher_texts)):
                    if b!=1:
                        p1 = plain_texts[ix][-2*(b):-2*(b-1)]
                        c1 = cipher_texts[ix][-2*(b):-2*(b-1)]
                    else:
                        p1 = plain_texts[ix][-2*(b):]
                        c1 = cipher_texts[ix][-2*(b):]
                        
                    p2 = plain_texts[ix][-2*(b+1):-2*(b)]
                    c2 = cipher_texts[ix][-2*(b+1):-2*(b)]
                    
                    coef1 = pair_to_poly(p1)
                    coef2 = pair_to_poly(p2)
                    # E
                    coef1 = power_poly(coef1,e1)
                    coef2 = power_poly(coef2,e2)
                    # A
                    coef1 = add_poly(mult_poly(coef2,a21), mult_poly(coef1,a11))
                    coef2 = mult_poly(coef2,a22)
                    # E
                    coef1 = power_poly(coef1,e1)
                    coef2 = power_poly(coef2,e2)
                    # A
                    coef1 = add_poly(mult_poly(coef2,a21), mult_poly(coef1,a11))
                    coef2 = mult_poly(coef2,a22)
                    # E
                    coef1 = poly_to_pair(power_poly(coef1,e1))
                    coef2 = poly_to_pair(power_poly(coef2,e2))
                    
                    if coef2+coef1!=c2+c1:
                        flag = False
                        break
                    
                if flag:
                    soln[b-1]['a'].add((poly_to_int(a11)))
                    soln[b-1]['e'].add(e1)
                    soln[b]['a'].add((poly_to_int(a22)))
                    soln[b]['e'].add(e2)
    print(b)
    print(soln)

1
{0: {'a': {38}, 'e': {25}}, 1: {'a': {27}, 'e': {19}}, 2: {'a': set(), 'e': set()}, 3: {'a': set(), 'e': set()}, 4: {'a': set(), 'e': set()}, 5: {'a': set(), 'e': set()}, 6: {'a': set(), 'e': set()}, 7: {'a': set(), 'e': set()}}
2
{0: {'a': {38}, 'e': {25}}, 1: {'a': {27}, 'e': {19}}, 2: {'a': {11}, 'e': {48}}, 3: {'a': set(), 'e': set()}, 4: {'a': set(), 'e': set()}, 5: {'a': set(), 'e': set()}, 6: {'a': set(), 'e': set()}, 7: {'a': set(), 'e': set()}}
3
{0: {'a': {38}, 'e': {25}}, 1: {'a': {27}, 'e': {19}}, 2: {'a': {11}, 'e': {48}}, 3: {'a': {112}, 'e': {93}}, 4: {'a': set(), 'e': set()}, 5: {'a': set(), 'e': set()}, 6: {'a': set(), 'e': set()}, 7: {'a': set(), 'e': set()}}
4
{0: {'a': {38}, 'e': {25}}, 1: {'a': {27}, 'e': {19}}, 2: {'a': {11}, 'e': {48}}, 3: {'a': {112}, 'e': {93}}, 4: {'a': {12}, 'e': {74}}, 5: {'a': set(), 'e': set()}, 6: {'a': set(), 'e': set()}, 7: {'a': set(), 'e': set()}}
5
{0: {'a': {38}, 'e': {25}}, 1: {'a': {27}, 'e': {19}}, 2: {'a': {11}, 'e': {48}}, 3: